In [1]:
import pandas as pd

df1 = pd.read_csv('/content/drive/My Drive/labeledTrainData.tsv', delimiter="\t")
df1 = df1.drop(['id'], axis=1)
df1.head()

,sentiment,review
0,1,With all this stuff going down at the moment w...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,0,The film starts with a manager (Nicholas Bell)...
3,0,It must be assumed that those who praised this...
4,1,Superbly trashy and wondrously unpretentious 8...


In [2]:
df2 = pd.read_csv('/content/drive/My Drive/imdb_master.csv',encoding="latin-1")
df2.head()

,Unnamed: 0,type,review,label,file
0,0,test,Once again Mr. Costner has dragged out a movie...,neg,0_2.txt
1,1,test,This is an example of why the majority of acti...,neg,10000_4.txt
2,2,test,"First of all I hate those moronic rappers, who...",neg,10001_1.txt
3,3,test,Not even the Beatles could write songs everyon...,neg,10002_3.txt
4,4,test,Brass pictures (movies is not a fitting word f...,neg,10003_3.txt


In [3]:
df2 = df2.drop(['Unnamed: 0','type','file'],axis=1)
df2.columns = ["review","sentiment"]
df2.head()

,review,sentiment
0,Once again Mr. Costner has dragged out a movie...,neg
1,This is an example of why the majority of acti...,neg
2,"First of all I hate those moronic rappers, who...",neg
3,Not even the Beatles could write songs everyon...,neg
4,Brass pictures (movies is not a fitting word f...,neg


In [4]:
df2 = df2[df2.sentiment != 'unsup']
df2['sentiment'] = df2['sentiment'].map({'pos': 1, 'neg': 0})
df2.head()

,review,sentiment
0,Once again Mr. Costner has dragged out a movie...,0
1,This is an example of why the majority of acti...,0
2,"First of all I hate those moronic rappers, who...",0
3,Not even the Beatles could write songs everyon...,0
4,Brass pictures (movies is not a fitting word f...,0


In [5]:
df = pd.concat([df1, df2]).reset_index(drop=True)
df.head()

,sentiment,review
0,1,With all this stuff going down at the moment w...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,0,The film starts with a manager (Nicholas Bell)...
3,0,It must be assumed that those who praised this...
4,1,Superbly trashy and wondrously unpretentious 8...


In [6]:
df.tail()

,sentiment,review
74995,1,"Seeing as the vote average was pretty low, and..."
74996,1,"The plot had some wretched, unbelievable twist..."
74997,1,I am amazed at how this movie(and most others ...
74998,1,A Christmas Together actually came before my t...
74999,1,Working-class romantic drama from director Mar...


In [7]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()


def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

df['Processed_Reviews'] = df.review.apply(lambda x: clean_text(x))

In [9]:
df.head()

,sentiment,review,Processed_Reviews
0,1,With all this stuff going down at the moment w...,stuff go moment mj ive start listen music watc...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi...",classic war world timothy hines entertain film...
2,0,The film starts with a manager (Nicholas Bell)...,film start manager nicholas bell give welcome ...
3,0,It must be assumed that those who praised this...,must assume praise film greatest film opera ev...
4,1,Superbly trashy and wondrously unpretentious 8...,superbly trashy wondrously unpretentious 80 ex...


In [10]:
df.Processed_Reviews.apply(lambda x: len(x.split(" "))).mean()

128.51009333333334

In [11]:
%tensorflow_version 1.x

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df['Processed_Reviews'])
list_tokenized_train = tokenizer.texts_to_sequences(df['Processed_Reviews'])

maxlen = 130
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = df['sentiment']

print(X_t.shape)
print(X_t[0])
print(y.shape)

embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

TensorFlow 1.x selected.


Using TensorFlow backend.


(75000, 130)
[   6  136  206   20  250 3669    1  702  276    4  116  340   86  778
  120 3483 1161  354  863  598  875  712 1466   51  287   29  595  664
  863   15 3837   51   28   26 3061  712  477  193  603  132    1    1
  524   41    7   94  377 1666  138 1651 1418  354   30   70  130 4221
 3787   39    4   29  354  541   70  603   50    5  153  179  534  138
  668  954 1190  427   23    1    1 1192  134    2   28    7    5  480
   77  938   19   28  533  436  163   61   55   34  576 3174 1377    2
   98  416 1514  293    5  929   28   58 1329 1095 2135   22  589  499
   34  583   22   26   28  192  444  369   26   99  277  491  250  303
   80    5  301 1448]
(75000,)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape             

In [12]:
batch_size = 100
epochs = 3
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)


Train on 60000 samples, validate on 15000 samples
Epoch 1/3
60000/60000 [==============================] - 102s 2ms/step - loss: 0.3504 - accuracy: 0.8410 - val_loss: 0.2221 - val_accuracy: 0.9204
Epoch 2/3
60000/60000 [==============================] - 99s 2ms/step - loss: 0.2204 - accuracy: 0.9134 - val_loss: 0.2790 - val_accuracy: 0.8904
Epoch 3/3
60000/60000 [==============================] - 99s 2ms/step - loss: 0.1693 - accuracy: 0.9369 - val_loss: 0.1642 - val_accuracy: 0.9480


In [13]:
df_test=pd.read_csv("/content/drive/My Drive/testData.tsv",header=0, delimiter="\t", quoting=3)
df_test.head()

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."


In [14]:
df_test=pd.read_csv("/content/drive/My Drive/testData.tsv",header=0, delimiter="\t", quoting=3)
df_test.head()

df_test["review"]=df_test.review.apply(lambda x: clean_text(x))

df_test["sentiment"] = df_test["id"].map(lambda x: 1 if int(x.strip('"').split("_")[1]) >= 5 else 0)
y_test = df_test["sentiment"]

list_sentences_test = df_test["review"]

list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

prediction = model.predict(X_te)
y_pred = (prediction > 0.5)

loss, accuracy = model.evaluate(X_te, y_test, batch_size=100)
print("accuracy", accuracy)

from sklearn.metrics import f1_score, confusion_matrix
# print('F1-score: {0}'.format(f1_score(y_pred, y_test)))
print('Confusion matrix:')
confusion_matrix(y_pred, y_test)

25000/25000 [==============================] - 9s 362us/step
accuracy 0.9589599967002869
Confusion matrix:


array([[12113,   639],
       [  387, 11861]])

In [15]:
df_test_review=pd.read_csv("/content/testreview.csv",header=0, delimiter="\t", quoting=3)
df_test_review.head()

,review
0,This is the best movie I have ever seen.
1,this is the worst. I mean how can someone come...


In [16]:
df_test_review=pd.read_csv("/content/testreview.csv",header=0, delimiter="\t", quoting=3)
df_test_review.head()

df_test_review["review"]=df_test_review.review.apply(lambda x: clean_text(x))

list_sentences_test_review = df_test_review["review"]
list_tokenized_test_review = tokenizer.texts_to_sequences(list_sentences_test_review)
X_review = pad_sequences(list_tokenized_test_review, maxlen=maxlen)

pred = model.predict(X_review)
f = []
for p in pred:
  f.append(1 if (p > 0.5) else 0)

print(f)

[1, 0]
